In [1]:
'''
trainだけだとテストユーザに抜け漏れがあったので、
trainとpriorを結合し、その中のorder_numberが一番新しい(=ユーザごとの最新の注文)ものだけをとってきて、
そのnumberで注文した全商品をテストでも再購入するとした(baseline)

'''

'\ntrainだけだとテストユーザに抜け漏れがあったので、\ntrainとpriorを結合し、その中のorder_numberが一番新しい(=ユーザごとの最新の注文)ものだけをとってきて、\nそのnumberで注文した全商品をテストでも再購入するとした(baseline)\n\n'

In [2]:
# main module
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas_profiling as pdp
import numpy as np

import datetime
import logging
import sys, os
sys.path.append('../src/') #モジュールが入っているディレクトリのパスを指定

import config
import maprepro as mpre
from utils import setup_logger, ModelFactory


In [3]:
IDIR = '../../../input/'
# priors = pd.read_csv(IDIR + 'order_products__prior.csv')
# train = pd.read_csv(IDIR + 'order_products__train.csv')
orders = pd.read_csv(IDIR + 'orders.csv')
products = pd.read_csv(IDIR + 'products.csv')
# sample = pd.read_csv(IDIR + 'sample_submission.csv')

# print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
# print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
# print('train {}:    {}'.format(train.shape, ', '.join(train.columns)))


In [6]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [5]:
print('optimize memory')
orders.order_dow = orders.order_dow.astype(np.int8)
orders.order_hour_of_day = orders.order_hour_of_day.astype(np.int8)
orders.order_number = orders.order_number.astype(np.int16)
orders.order_id = orders.order_id.astype(np.int32)
orders.user_id = orders.user_id.astype(np.int32)
orders.days_since_prior_order = orders.days_since_prior_order.astype(np.float32)

# products.drop(['product_name'], axis=1, inplace=True)
products.aisle_id = products.aisle_id.astype(np.int8)
products.department_id = products.department_id.astype(np.int8)
products.product_id = products.product_id.astype(np.int32)

train.reordered = train.reordered.astype(np.int8)
train.add_to_cart_order = train.add_to_cart_order.astype(np.int16)


priors.order_id = priors.order_id.astype(np.int32)
priors.add_to_cart_order = priors.add_to_cart_order.astype(np.int16)
priors.reordered = priors.reordered.astype(np.int8)
priors.product_id = priors.product_id.astype(np.int32)

optimize memory


In [6]:
mpre.missing_values_table(priors)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [7]:
mpre.missing_values_table(train)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [8]:
mpre.missing_values_table(orders)
# mpre.missing_values_table(sample)

Your selected dataframe has 7 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
days_since_prior_order,206209,6.0


In [9]:
mpre.missing_values_table(products)

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [10]:
priors

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [11]:
train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [12]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [13]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [15]:
test_orders = orders[orders.eval_set == 'test']
train_orders = orders[orders.eval_set == 'train']
prior_orders = orders[orders.eval_set == 'prior']

In [16]:
trainn = train.merge(train_orders, how='inner', on='order_id')
priorss = priors.merge(prior_orders, how='inner', on='order_id')

In [17]:
priorss

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0
...,...,...,...,...,...,...,...,...,...,...
32434484,3421083,39678,6,1,25247,prior,24,2,6,21.0
32434485,3421083,11352,7,0,25247,prior,24,2,6,21.0
32434486,3421083,4600,8,0,25247,prior,24,2,6,21.0
32434487,3421083,24852,9,1,25247,prior,24,2,6,21.0


In [18]:
trainn

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,train,4,4,10,9.0
1,1,11109,2,1,112108,train,4,4,10,9.0
2,1,10246,3,0,112108,train,4,4,10,9.0
3,1,49683,4,0,112108,train,4,4,10,9.0
4,1,43633,5,1,112108,train,4,4,10,9.0
...,...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,3,1,169679,train,30,0,10,4.0
1384613,3421063,35548,4,1,169679,train,30,0,10,4.0
1384614,3421070,35951,1,1,139822,train,15,6,10,8.0
1384615,3421070,16953,2,1,139822,train,15,6,10,8.0


In [27]:
all_df = pd.concat([trainn,priorss],axis=0)
tmp = ['order_dow','order_hour_of_day','days_since_prior_order']
all_df = all_df.drop(columns=tmp)

In [29]:
# all_df.sort_values(["user_id","order_number"],ascending=[True,False]).head(50)
max_order = all_df.groupby(["user_id"]).agg({'order_number': 'max'})
max_order = max_order.rename(columns={'order_number': 'max_order_number'})
all_df = all_df.merge(max_order, how='inner', on='user_id')
all_df = all_df.loc[all_df['order_number'] == all_df['max_order_number']]
all_df = all_df.drop(columns='max_order_number')

In [59]:
all_df.product_id = all_df.product_id.astype(str)

d= dict()
# j=0
for row in all_df.itertuples():
    # if j==10:
    #     break
    u_id = row.user_id
    u_pro = row.product_id
    if u_id not in d:
        d[u_id] = (u_pro)
    else:
        d[u_id] = d[u_id]+' '+ u_pro
    # j+=1

In [60]:
d = pd.DataFrame.from_dict(d, orient='index') # orient=向き、キーをインデックスに使うかカラムに使うか
d.columns = ['all_products']
userXproduct = d

In [81]:
userXproduct.index.name="user_id"
userXproduct.reset_index(inplace=True)
userXproduct

,user_id,all_products
0,112108,49302 11109 10246 49683 43633 13176 47209 22035
1,79431,39612 19660 49235 43086 46620 34497 48679 46979
2,42756,11913 18159 4461 21616 23622 32433 28842 42625...
3,17227,20574 30391 40706 25610 27966 24489 39275
4,56463,8859 19731 43654 13176 4357 37664 34065 35951 ...
...,...,...
206204,82688,13176 29820 21137 11140 5785 13740 47766 43789...
206205,143211,7862 33768 21137 6184
206206,4689,432 39032 27429 22893 49637 41290 9844 19220 4...
206207,9295,5068 21903 39275 20940 1940 19767


In [120]:
sample = pd.read_csv(IDIR + 'sample_submission.csv'); sample.drop('products',axis=1, inplace=True)
sample = sample.merge(orders[['order_id','user_id']],how='inner',on='order_id')
sample = sample.merge(userXproduct,how='inner',on='user_id')
tmp=['user_id']; sample.drop(tmp,axis=1, inplace=True)
sample.columns=['order_id','products']
sample


,order_id,products
0,17,13107 26429 39275 21463
1,34,44912 47766 16083 2596 37630 23028 39475 13176...
2,137,13740 34126 44422 29594 25890 24852 41787 2379...
3,182,37642 5621 5479 47672 9337 39275 32109 40377 1...
4,257,49235 24852 30233 29837 37646 27104 47547 3947...
...,...,...
74995,3420740,2314 1158 48070 28476 19660 17872 35951 12597 ...
74996,3420877,3661 8136 32293 42450 13646 12238 5161 49111 2...
74997,3420888,43961 46906 10880 35951 44632 10111 20542
74998,3420989,46676 35004 5134 45866 42450 17948 39075 8555 ...


In [121]:
submit=sample
submit.to_csv('../../output/baseline_submit.csv',index=False)

In [122]:
p = pd.read_csv('../../output/baseline_submit.csv')
mpre.missing_values_table(p)


Your selected dataframe has 2 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values
